In [3]:
s = "1"

int(s) == 2


False

In [1]:
# 修改checkpoint路径
ckpt_dir = "/home/hanlv/workspace/code/research/infodemic/LLM/swift/examples/pytorch/llm/output/openchat_3.5/with_solar_info/brave/data1-split=8:2-ratio=1.0/adalora/lr=3.3e-4-20240215-20:05:01/checkpoint-609"

import os
import sys
import json
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

dirs = [".."]
for _dir in dirs:
    if _dir not in sys.path:
        sys.path.append(_dir)

from swift.llm import (
    get_model_tokenizer, get_template, inference
)
from swift.tuners import Swift
from custom import CustomModelType, CustomTemplateType
import evaluation

with open(f"{ckpt_dir}/sft_args.json", "r") as f:
    sft_args = json.load(f)

def get_model_template():
    model_type, template_type = sft_args["model_type"], sft_args["template_type"]
    model, tokenizer = get_model_tokenizer(
        model_type, model_kwargs={'device_map': 'auto'},
        model_dir=sft_args["model_cache_dir"]
    )
    model = Swift.from_pretrained(model, ckpt_dir, inference_mode=True)
    if sft_args["sft_type"] == 'adalora':
        model = model.to(model.dtype)
    model.generation_config.max_new_tokens = 512
    model.generation_config.do_sample = False

    template = get_template(template_type, tokenizer)

    return model, template

model, template = get_model_template()



2024-02-16 01:25:26,550 - modelscope - INFO - PyTorch version 2.1.2 Found.
2024-02-16 01:25:26,552 - modelscope - INFO - Loading ast index from /home/hanlv/.cache/modelscope/ast_indexer
2024-02-16 01:25:26,670 - modelscope - INFO - Loading done! Current index file version is 1.11.0, with md5 dc3cda2b56a86b0242cd145399a14071 and a total number of 953 components indexed
[INFO:swift] Setting torch_dtype: torch.bfloat16
[INFO:swift] model_config: MistralConfig {
  "_name_or_path": "/home/css/models/openchat-3.5-0106",
  "architectures": [
    "MistralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 32000,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "rms_norm_eps": 1e-05,
  "rope_theta": 10000.0,
  "sliding_window": 4096,
  "tie_word_embeddings": fa

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [21]:
items = [
    {"query": "According to the CLAIM and the PRIOR KNOWLEDGE, please classify the CLAIM as TRUE or FALSE. If the content described by the CLAIM is correct, then classify it as TRUE; if the content described by the CLAIM is incorrect, then classify it as FALSE.\n\nCLAIM:\nPublication date: 2020-01-31\nContent: Yes, suspected case of coronavirus reported in Kenya, patient in hospital after showing symptoms\n\nPRIOR KNOWLEDGE:\nInformation 1:\nPublication date: None\nTitle: Yes, suspected case of coronavirus reported in Kenya, patient ...\nContent:\nNTV reported that a Kenyan student, a passenger on a plane from Guangzhou city in China to Nairobi’s Jomo Kenyatta International Airport, had shown symptoms of the virus. The student was admitted to Kenyatta National Hospital, the country’s biggest referral hospital, on the morning of 28 January, NTV said.\nOn 28 January Kenya’s ministry of health issued a statement on its Facebook and Twitter pages confirming that a patient showing coronavirus symptoms had been admitted to hospital. “On the morning of 28 January 2020, the Ministry of Health was notified of a passenger aboard flight KQ 887 from Guangzhou China who upon screening on arrival at Jomo Kenyatta International Airport (JKIA) was found to have fever,” the statement reads.\n“He was immediately isolated in a temporary holding room at JKIA and later transferred to the Infectious Disease Unit, Kenyatta National Hospital (KNH) for investigation and admission.” A day later the ministry issued an update, saying the patient would remain in isolation until tests were completed.\nInformation 2:\nPublication date: None\nTitle: Yes, suspected case of coronavirus reported in Kenya, patient in ...\nContent:\nNTV reported that a Kenyan student, a passenger on a plane from Guangzhou city in China to Nairobi’s Jomo Kenyatta International Airport, had shown symptoms of the virus. The student was admitted to Kenyatta National Hospital, the country’s biggest referral hospital, on the morning of 28 January, NTV said.\nOn 28 January Kenya’s ministry of health issued a statement on its Facebook and Twitter pages confirming that a patient showing coronavirus symptoms had been admitted to hospital. “On the morning of 28 January 2020, the Ministry of Health was notified of a passenger aboard flight KQ 887 from Guangzhou China who upon screening on arrival at Jomo Kenyatta International Airport (JKIA) was found to have fever,” the statement reads.\n“He was immediately isolated in a temporary holding room at JKIA and later transferred to the Infectious Disease Unit, Kenyatta National Hospital (KNH) for investigation and admission.” A day later the ministry issued an update, saying the patient would remain in isolation until tests were completed.\nInformation 3:\nPublication date: None\nTitle: Coronavirus Disease 2019 (COVID-19) in Kenya: Preparedness, response ...\nContent:\nThe world and <strong>Kenya</strong> face a potential pandemic as the respiratory virus <strong>Coronavirus</strong> Disease 2019 (COVID-19) affects world populations. Nations have been forced to intervene and issue directions under executive orders to ensure the pandemic is contained. ...\nInformation 4:\nPublication date: None\nTitle: MoH | Ministry of Health\nContent:\nThe Kenya Medical Training College (KMTC) is a State Corporation established through an Act of Parliament under the Ministry of Health, entrusted with the role of training various disciplines in the health sector to serve the Kenyan interests, East Africa and beyond.\nMalaria vaccination targeting children within Kenya's lake-endemic region\nInformation 5:\nPublication date: 2020-03-13\nTitle: Kenya Confirms First COVID-19 Infection\nContent:\nEast African nation becomes 11th country in Africa to confirm the <strong>coronavirus</strong>\n\nTo classify the CLAIM accurately, let's analyze the available information and relate it to the CLAIM.\n\nCLAIM: Publication date: 2020-01-31, Content: Yes, suspected case of coronavirus reported in Kenya, patient in hospital after showing symptoms.\n\nInformation 1 and Information 2 both provide details about a Kenyan student who showed symptoms of the virus after arriving from Guangzhou, China. The student was admitted to the Kenyatta National Hospital for investigation. The Kenyan Ministry of Health confirmed this event and mentioned that the patient was in isolation until tests were completed.\n\nInformation 3 discusses the potential pandemic caused by the coronavirus (COVID-19) and how nations, including Kenya, are preparing and responding to it. This information supports the presence of coronavirus concerns in Kenya.\n\nInformation 4 is not directly related to the CLAIM but provides background information about the Kenya Medical Training College and the Ministry of Health's role in the health sector.\n\nInformation 5 reports that Kenya confirmed its first COVID-19 infection on March 13, 2020. This information shows that the coronavirus did indeed reach Kenya, although it is not directly related to the specific suspected case mentioned in the CLAIM.\n\nBased on Information 1, Information 2, and Information 3, it can be concluded that the CLAIM is accurate. The information provided confirms the presence of a suspected coronavirus case in Kenya, with the patient being hospitalized after showing symptoms. Therefore, the CLAIM can be classified as TRUE.", "response": "FALSE."}
	
]
_id = 0

query = items[_id]["query"]
# query = 'what is 1+1?'
print(query + '\n')
response, history = inference(
    model, template, query, 
    verbose=False, prompt_prefix='', output_prefix=''
)

print(f'response: {response} label: {items[_id]["response"]}')


According to the CLAIM and the PRIOR KNOWLEDGE, please classify the CLAIM as TRUE or FALSE. If the content described by the CLAIM is correct, then classify it as TRUE; if the content described by the CLAIM is incorrect, then classify it as FALSE.

CLAIM:
Publication date: 2020-01-31
Content: Yes, suspected case of coronavirus reported in Kenya, patient in hospital after showing symptoms

PRIOR KNOWLEDGE:
Information 1:
Publication date: None
Title: Yes, suspected case of coronavirus reported in Kenya, patient ...
Content:
NTV reported that a Kenyan student, a passenger on a plane from Guangzhou city in China to Nairobi’s Jomo Kenyatta International Airport, had shown symptoms of the virus. The student was admitted to Kenyatta National Hospital, the country’s biggest referral hospital, on the morning of 28 January, NTV said.
On 28 January Kenya’s ministry of health issued a statement on its Facebook and Twitter pages confirming that a patient showing coronavirus symptoms had been admitt